# Analysis!

In [ ]:
import sqlite3 as sql
import pandas as pd
import csv

# sklearn k means clustering?

Grab dataframe

In [ ]:
conn = sql.connect('data/choice.db')
query = 'SELECT * FROM prediction'
df = pd.read_sql_query(query, conn)

conn.close()

Donation needs to be ordered by date, consider date format.